In [1]:
import torch
import glob
import tqdm

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from utilities import utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from torch.utils.data import TensorDataset
from pythae.models import VAE, VAEConfig
from pythae.trainers import BaseTrainerConfig
from pythae.pipelines.training import TrainingPipeline
from pythae.models.nn import BaseEncoder, BaseDecoder
from pythae.models.base.base_utils import ModelOutput

In [2]:
paths = sorted(glob.glob('./data/environmental_embeddings_0001/0001/*.msgpack'))

X = []
for path in tqdm.tqdm(paths):
    pos_emb, neg_emb = utils.read_embedding_data(path)
    X.append(pos_emb)
X = np.concatenate(X, axis=0)

# reshape to 2D to apply L2 normalization
X = X.reshape(-1, 768)
X, l2_norm = normalize(X, norm='l2', axis=1, return_norm=True)

# reshape it back
X = X.reshape(-1, 77, 768)

Xtr, Xvl = train_test_split(X, test_size=0.2, random_state=42)

train_data = torch.tensor(Xtr, dtype=torch.float32)
val_data = torch.tensor(Xvl, dtype=torch.float32)

train_data = F.pad(train_data, (0, 0, 1, 2))
val_data = F.pad(val_data, (0, 0, 1, 2))

train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)

100%|██████████| 1000/1000 [00:06<00:00, 147.61it/s]


In [3]:
class Encoder1DCNN(BaseEncoder):
    def __init__(self, args=None):
        BaseEncoder.__init__(self)
        self.seq_len, self.input_dim = args.input_dim
        self.encoder_layers = nn.Sequential(
            nn.Conv1d(self.input_dim, 512, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv1d(512, 256, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            # nn.MaxPool1d(kernel_size=3, stride=1),
            nn.Conv1d(256, 128, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            # nn.MaxPool1d(kernel_size=3, stride=1),
            nn.Conv1d(128, 64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            # nn.MaxPool1d(kernel_size=3, stride=1),
            nn.Conv1d(64, 32, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            # nn.MaxPool1d(kernel_size=3, stride=1)
        )

        self.fc_mu = nn.Linear(2560, args.latent_dim)
        self.fc_logvar = nn.Linear(2560, args.latent_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.encoder_layers(x)
        x = x.view(x.size(0), -1)

        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)

        output = ModelOutput(embedding=mu, log_covariance=logvar)

        return output
    
class Decoder1DCNN(BaseDecoder):
    def __init__(self, args=None):
        BaseDecoder.__init__(self)
        self.seq_len, self.input_dim = args.input_dim
        self.fc = nn.Linear(args.latent_dim, 2560)
        self.decoder_layers = nn.Sequential(
            nn.ConvTranspose1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            # nn.Upsample(scale_factor=2),
            nn.ConvTranspose1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            # nn.Upsample(scale_factor=2),
            nn.ConvTranspose1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            # nn.Upsample(scale_factor=2),
            nn.ConvTranspose1d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(512, self.input_dim, kernel_size=3, stride=1, padding=1),
            # nn.Upsample(scale_factor=2),
            nn.Tanh()
        )

    def forward(self, z):
        x = self.fc(z)
        x = x.view(x.size(0), 32, 80)
        x = self.decoder_layers(x)
        x = x.permute(0, 2, 1)

        output = ModelOutput(
            reconstruction=x
        )

        return output

In [4]:
config = BaseTrainerConfig(
    output_dir='./experiments/vae_1DCNN',
    learning_rate=1e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_epochs=500,
    optimizer_cls="AdamW",
    optimizer_params={"weight_decay": 0.05, "betas": (0.91, 0.99)}
)

model_config = VAEConfig(
    input_dim=(77, 768),
    latent_dim=1024
)

encoder = Encoder1DCNN(model_config)
decoder = Decoder1DCNN(model_config)

model = VAE(
    model_config=model_config,
    encoder=encoder,
    decoder=decoder
)

In [5]:
pipeline = TrainingPipeline(
    training_config=config,
    model=model
)

In [6]:
pipeline(
    train_data=train_data,
    eval_data=val_data
)

Preprocessing train data...
Checking train dataset...
Preprocessing eval data...

Checking eval dataset...
Using Base Trainer

Model passed sanity check !
Ready for training.

Created ./experiments/vae_1DCNN/VAE_training_2023-10-20_19-48-40. 
Training config, checkpoints and final model will be saved here.

Training params:
 - max_epochs: 500
 - per_device_train_batch_size: 128
 - per_device_eval_batch_size: 128
 - checkpoint saving every: None
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.91, 0.99)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.05
)
Scheduler: None

Successfully launched training !



Training of epoch 1/500:   0%|          | 0/7 [00:00<?, ?batch/s]

Eval of epoch 1/500:   0%|          | 0/2 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
Train loss: 40.9425
Eval loss: 36.1142
--------------------------------------------------------------------------


Training of epoch 2/500:   0%|          | 0/7 [00:00<?, ?batch/s]

...

Eval of epoch 499/500:   0%|          | 0/2 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
Train loss: 20.9191
Eval loss: 20.8219
--------------------------------------------------------------------------


Training of epoch 500/500:   0%|          | 0/7 [00:00<?, ?batch/s]

Eval of epoch 500/500:   0%|          | 0/2 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
Train loss: 20.8938
Eval loss: 20.8073
--------------------------------------------------------------------------
Training ended!
Saved final model in ./experiments/vae_1DCNN/VAE_training_2023-10-20_19-48-40/final_model


In [7]:
ckpt = torch.load('./experiments/vae_1DCNN/VAE_training_2023-10-20_19-48-40/final_model/model.pt')
model = VAE(
    model_config=model_config,
    encoder=encoder,
    decoder=decoder
)

model.load_state_dict(ckpt['model_state_dict'])
model.eval()

VAE(
  (decoder): Decoder1DCNN(
    (fc): Linear(in_features=1024, out_features=2560, bias=True)
    (decoder_layers): Sequential(
      (0): ConvTranspose1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
      (2): ConvTranspose1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (3): ReLU()
      (4): ConvTranspose1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (5): ReLU()
      (6): ConvTranspose1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (7): ReLU()
      (8): ConvTranspose1d(512, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (9): Tanh()
    )
  )
  (encoder): Encoder1DCNN(
    (encoder_layers): Sequential(
      (0): Conv1d(768, 512, kernel_size=(3,), stride=(1,), padding=same)
      (1): ReLU()
      (2): Conv1d(512, 256, kernel_size=(3,), stride=(1,), padding=same)
      (3): ReLU()
      (4): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=same)
      (5): ReLU()
      (6): Conv1d(128, 64, k

In [10]:
with torch.no_grad():
    reconstruction = model.reconstruct(train_data.to('cuda'))

recon = reconstruction[:, 1:78, :].cpu()
val = val_data[:, 1:78, :]
train = train_data[:, 1:78, :]

In [12]:
criterion = torch.nn.MSELoss(reduction='sum')
criterion(train, recon) / 800

tensor(41.7904)